In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils as utils
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# set seed
torch.manual_seed(42)
np.random.seed(42)


In [2]:
from typing import Tuple, List, Callable
from torch.distributions import Distribution
from sklearn.preprocessing import StandardScaler
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
from sklearn import metrics
from tqdm import tqdm
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from scipy.stats import gaussian_kde
from IPython.display import clear_output


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# https://sebastiancallh.github.io/post/affine-normalizing-flows/
# uses Real NVP paper

# Define the Normalising Flow model template
class NormalisingFlow(nn.Module):

    def __init__(self, latent: Distribution, flows: List[nn.Module]):
        super(NormalisingFlow, self).__init__()
        self.latent = latent
        self.flows = nn.ModuleList(flows)

    def latent_log_prob(self, z: torch.Tensor) -> torch.Tensor:
        log_prob_z = self.latent.log_prob(z)
        return log_prob_z

    def latent_sample(self, num_samples: int = 1) -> torch.Tensor:
        return self.latent.sample((num_samples,))

    def sample(self, num_samples: int = 1) -> torch.Tensor:
        '''Sample a new observation x by sampling z from
        the latent distribution and pass through g.'''
        return self.g(self.latent_sample(num_samples))

    def f(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        '''Maps observation x to latent variable z.
        Additionally, computes the log determinant
        of the Jacobian for this transformation.
        Inverse of g.'''
        #z, sum_log_abs_det = x, torch.zeros(x.size(0)).to(device) # change to x device
        
        z, sum_log_abs_det = x, torch.zeros(x.size(0), device = x.device)
        for flow in self.flows: # for each transformation in the flow
            z, log_abs_det = flow.f(z)
            sum_log_abs_det += log_abs_det

        return z, sum_log_abs_det

    def g(self, z: torch.Tensor) -> torch.Tensor:
        '''Maps latent variable z to observation x.
        Inverse of f.'''
        with torch.no_grad():
            x = z
            for flow in reversed(self.flows):
                x = flow.g(x)

        return x

    def g_steps(self, z: torch.Tensor) -> List[torch.Tensor]:
        '''Maps latent variable z to observation x
        and stores intermediate results.'''
        xs = [z]
        for flow in reversed(self.flows):
            xs.append(flow.g(xs[-1]))

        return xs

    def log_prob(self, x: torch.Tensor) -> torch.Tensor:
        '''Computes log p(x) using the change of variable formula.'''
        z, log_abs_det = self.f(x)
        log_prob_x = self.latent_log_prob(z) + log_abs_det
        return log_prob_x

    def __len__(self) -> int:
        return len(self.flows)


class AffineCouplingLayer(nn.Module):

    def __init__(self, theta: nn.Module, split: Callable[[torch.Tensor], Tuple[torch.Tensor, torch.Tensor]]):
        super(AffineCouplingLayer, self).__init__()
        self.theta = theta
        self.split = split

    def f(self, x: torch.Tensor) -> torch.Tensor:
        '''f: x -> z. The inverse of g.'''
        x2, x1 = self.split(x) # flip the split #insert other permutation for 3D+ data
        ### permutation at one point
        t, s = self.theta(x1)
        # Tau coupling function: e^s + t
        z1, z2 = x1, x2 * torch.exp(s) + t
        log_det = s.sum(-1) # sum over the last dimension
        return torch.cat((z1, z2), dim=-1), log_det

    def g(self, z: torch.Tensor) -> torch.Tensor:
        '''g: z -> x. The inverse of f.'''
        z1, z2 = self.split(z)
        t, s = self.theta(z1)
        x1, x2 = z1, (z2 - t) * torch.exp(-s)
        return torch.cat((x2, x1), dim=-1)


class Conditioner(nn.Module):
    'The conditioner is the Neural Network that helps fit the model to the data by learning theta_i = (s_i,t_i)'

    def __init__(self, in_dim: int, out_dim: int, num_hidden: int, hidden_dim: int, num_params: int):
        super(Conditioner, self).__init__()
        self.input = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),  
            nn.LeakyReLU(inplace=True)  
        )
        self.hidden = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),  
                nn.Dropout(0.3),
                nn.LeakyReLU(inplace=True)  
            )
            for _ in range(num_hidden)
        ])

        self.num_params = num_params
        self.out_dim = out_dim
        self.output = nn.Linear(hidden_dim, out_dim * num_params)
        # initialisere output lag conditioner alle vægte og bias til 0
        nn.init.zeros_(self.output.weight)
        nn.init.zeros_(self.output.bias) 

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.input(x)
        for h in self.hidden:
            x = h(x)

        batch_params = self.output(x).reshape(x.size(0), self.out_dim, -1)
        #batch_params[:,:,1] *= 0.001
        #batch_params[:,:,0] *= 0.001 
        params = batch_params.chunk(self.num_params, dim=-1)
        return [p.squeeze(-1) for p in params]

class CNN(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),

            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) 
        )

        self.linear_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, latent_dim),
            nn.BatchNorm1d(latent_dim)
            # nn.Dropout(p=0.5),
            # nn.Linear(128*7*7, 128),
            # nn.BatchNorm1d(128),
            # nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
            # nn.Linear(128, 64),
            # nn.BatchNorm1d(64),
            # nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
            # nn.Linear(64, 10)
        )

        # self.flat = nn.Flatten()
        # # I want the feature space to be 2D?? eller 10 for num_classe
        # self.fc1 = nn.Linear(32*8*8, latent_dim) #num_classes
        # self.batchnorm1 = nn.BatchNorm1d(latent_dim) #num_classes

    def forward(self, x):
        x = self.conv_block(x)
        #print("Shape after conv_block:", x.shape) #64batchsize x 64channels x 1height x 1width
        #x = x.view(x.size(0), -1) # flatten
        x = self.linear_block(x)
        return x
    
  ## Afprøve:
# learning rate scheduler
# rapport
# Langt tid senere: korrekte usikkerheder senere
# plot i datarummet - meshgrid - normaliseret alpha eller flows    

In [5]:
class PosteriorNetwork(nn.Module):
    def __init__(self, latent_dim: int, flow_models: List[nn.Module], N: torch.tensor, num_classes: int, y: torch.Tensor):
        super(PosteriorNetwork, self).__init__()
        self.cnn = CNN(latent_dim)
        self.flow_models = nn.ModuleList(flow_models)
        self.N = N
        self.y = y
        self.num_classes = num_classes 
    
    def forward(self, x, N):
        batch_size = x.size(0)
        # N is number of inputs in each class total
        z = self.cnn(x) 
        # for each class, since outputdim = num_classes
        alpha = torch.zeros((batch_size, self.num_classes)).to(z.device.type)
        log_q = torch.zeros((batch_size, self.num_classes)).to(z.device.type)
        beta_i = torch.zeros((batch_size, self.num_classes)).to(z.device.type)

        # for each class, compute 
        for c in range(self.num_classes):
            log_prob = self.flow_models[c].log_prob(z) #P(z|c,phi)
            log_q[:,c] = log_prob
            beta_prior = 1 
            beta_i[:,c] = N[c] * torch.exp(log_prob)  #formula (4) from paper
            alpha[:,c] = beta_prior + beta_i[:,c] #or just beta_i[c]?

        # grad_loss
        #loss = self.loss_postnet(alpha, y, batch_size)

        ##alpha = F.normalize(alpha, p=1, dim=1) # to get p^bar_c which is the average of alphas
        #preds = self.predict(alphas) #categorical prediction using argmax on p_bar_c
        return alpha

    def loss_postnet(self, alpha, y, batch_size): #UCE loss 
        #alpha is the p vector with alphas for each class
        #y_hot is the ground-truth class labels
        alpha_0 = torch.sum(alpha, dim=1, keepdim=True) #batch x 1
        digamma_alpha_c = torch.digamma(alpha[range(batch_size),y]) # batch x 1

        digamma_alpha_0 = torch.digamma(alpha_0) # batch x 1, hver obs får sin egent logprobs
        uce_loss = digamma_alpha_c - digamma_alpha_0 #elementwise
        #uncertain_loss = torch.sum((digamma_alpha_c - digamma_alpha_sum) * alpha, dim=1)
        
        # entropy in Dirichlet distribution
        uce_loss = -torch.mean(uce_loss) #negative since we want to minimize the loss
        #approximates the true posterior distribution for the categorical distribution p
        return uce_loss

In [6]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
batch_size = 64 # standard value
from torch.utils.data.dataset import Subset
subset_percentage = 0.1

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

num_samples = int(len(train_dataset) * subset_percentage)
print("Number of samples:", num_samples)
train_subset = Subset(train_dataset, range(num_samples*0.8))
test_subset = Subset(test_dataset, range(num_samples*0.2)) 

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

# Get ground-truth label counts N_c
# Dictionary with class names from indexes.
classes = {index: name for name, index in train_dataset.class_to_idx.items()}
print(classes)

# Initialise dictionary to store the counts for each class using class indexes
N = {index: 0 for index in range(len(classes))}
# Count the occurrences of each class
for _, target in train_dataset:
    N[target] += 1
N = torch.tensor([N[index] for index in range(len(classes))])
print(N)

y_train = torch.tensor([target for _, target in train_dataset])
print(y_train)

Number of samples: 6000


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
def image_show(img):
    img = img.detach().cpu()
    img = img / 2 + 0.5   #Unnormalise
    with sns.axes_style("white"):
        plt.figure(figsize=(8, 8))
        plt.imshow(img.permute((1, 2, 0)).numpy())
        plt.axis('off')
        plt.show()

def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

def init_weights(model):
    if isinstance(model, nn.Linear):
        nn.init.xavier_normal_(model.weight)
        nn.init.zeros_(model.bias) 

In [ ]:
# Get random training images and show them.
images, labels = next(iter(train_loader))
image_show(utils.make_grid(images))

In [ ]:
num_classes = 10
latent_dim = 2 # the encoder outputs 2D latent space
data_dim = 2 # the encoder outputs 2D latent space
in_dim= data_dim // 2 # since we split the data
out_dim= data_dim // 2
num_params = 2 # s and t
num_hidden = 2 # number of hidden layers
hidden_dim = 32 # neurons in hidden layers
num_flows = 4 # number of coupling flow layers

#lr = 0.0031
lr = 0.01
weight_decay = 1e-6  # L2 regularization strength to prevent overfitting in Adam or AdamW 
num_epochs = 10 # flere epochs maybe 12000
batch_size = 64
validation_every_steps = 1 # is actually every epoch in training loop!!
#validation_every_epochs = 5
early_stop_delta = 0.0001
early_stop_patience = 15
split = lambda x: x.chunk(2, dim=-1) #splits in half

# initalise the flows and postnet model
flow_models = []
for class_label in range(num_classes):
    conditioner = Conditioner(in_dim=in_dim, out_dim=out_dim, num_hidden=num_hidden, hidden_dim=hidden_dim, num_params=num_params)
    affine_coupling = AffineCouplingLayer(conditioner, split=split) # split the tensor into 2 parts
    flows = [affine_coupling for _ in range(num_flows)]
    latent_distribution = torch.distributions.MultivariateNormal(loc=torch.zeros(data_dim).to(device), scale_tril=torch.eye(data_dim).to(device)) #maybe move out of loop?

    flow_model = NormalisingFlow(latent_distribution, flows).to(device)
    #flow_model = NormalisingFlow(latent_distribution, flows).apply(init_weights).to(device)
    flow_models.append(flow_model)

postnet_model = PosteriorNetwork(latent_dim, flow_models, N, num_classes, y_train).to(device) 
optimiser = optim.AdamW(postnet_model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
############## TRAINING ######################
##############################################
def accuracy(y_train, preds):
    accuracy = accuracy_score(y_train.cpu().numpy(), preds.cpu().numpy())
    return accuracy

####### TO DO : #######
## Implement learning rate scheduling - adaptive learning rate during training: step_decay, learning rate annealing or learning rate scheduler
## - dividere med 10 på et tidspunkt

## Use a hyperparameter tuning library like Optuna or GridSearchCV to automate the search process.
## Cross-validation - implement a k-fold cross-validation to evaluate the model performance on the training data.
## Check normalisation of data
## Check regularization strength - e.g. weight decay or dropout
## Ensemble Methods: If applicable, consider using ensemble methods like bagging or boosting by training multiple models and combining their predictions.


def train(model, optimiser, train_loader, test_loader, num_epochs, validation_every_steps, early_stop_delta, early_stop_patience):
    model.train()
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []
    all_train_losses = []
    best_val_loss = float("Inf")
    step = 0 # how many batches we have trained on (each batch is 32 samples) #1600 training samples / 32 batch size = 50 batches per epoch
    counter = 0
    for epoch in range(num_epochs): #one forward pass through the entire training set

        train_losses_batches, train_accuracies_batches = [], []

        # batches_counter = 0
        for batch_index, (X_train, y_train) in enumerate(train_loader):
            X_train, y_train = X_train.to(device), y_train.to(device)
            
            # batches_counter += 1
            # print(f"Batch: {batches_counter}")
            # print("batch size: ", X_train.size(0))
            # Forward pass
            alpha = model(X_train, N)
            loss = model.loss_postnet(alpha, y_train, X_train.size(0)) #batch size
            # Perform one training step
            optimiser.zero_grad()
            loss.backward()
            #utils.clip_grad_norm_(flow_model.parameters(), max_norm=5.0)
            optimiser.step()
            step += 1
            #train_losses.append(loss.item())

            # Compute training accuracy and loss for this batch
            with torch.no_grad():
                preds = torch.max(alpha, dim=-1)[1]
                train_accuracies_batches.append(accuracy(y_train, preds))
                train_losses_batches.append(loss.item())
                #batch_accuracy = accuracy_score(y_train.cpu().numpy(), preds.cpu().numpy())
                #train_accuracies.append(batch_accuracy)
                all_train_losses.append(loss.item())

        if epoch % validation_every_steps == 0:
            train_losses.append(np.mean(train_losses_batches))
            train_accuracies.append(np.mean(train_accuracies_batches))

            val_losses_batches = []
            #val_accuracies_batches = []
            val_correct = []
            model.eval()
            with torch.no_grad():   
                for batch_index, (X_test, y_test) in enumerate(test_loader):
                    X_test, y_test = X_test.to(device), y_test.to(device)
                    # Evaluation Forward pass
                    alpha = model(X_test, N) # gives a vector with alphas for each class
                    loss = model.loss_postnet(alpha, y_test, X_test.size(0)) #gives a loss
                    
                    # Evaluation accuracy and loss for this batch
                    preds = torch.max(alpha, dim=-1)[1]
                    
                    correct_batch = (preds == y_test).sum().item()
                    val_correct.append(correct_batch)

                    #Maybe: Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    #val_accuracies_batches.append(accuracy(y_test, preds) * len(X_test))

                    # append the loss for this batch
                    val_losses_batches.append(loss.item())

            val_accuracy = sum(val_correct) / len(test_dataset) 
            val_accuracies.append(val_accuracy)

            val_loss = np.mean(val_losses_batches) 
            val_losses.append(val_loss)
            #val_losses.append(np.mean(val_losses_batches))
            #val_accuracies.append(np.sum(val_accuracies_batches) / len(test_dataset))   

            if val_losses[-1] < -1.:
                print("Unstable training")
                break
            if np.isnan(val_losses[-1]):
                print('Detected NaN Loss')
                break
            # If val_loss is the best so far, save the model state_dict and reset the early stopping counter
            if val_losses[-1] < best_val_loss:
                best_val_loss = val_losses[-1]
                counter = 0
                best_model = model.state_dict()
                torch.save({'epoch': epoch, 'model_state_dict': best_model, 'loss': best_val_loss}, 'best_model_mnist.pth')
                print('Model saved')

            # Early stopping - if val_loss is not improving (plus a delta e-4 as buffer) then start counter
            # after patience of a certain number of validations, then stop training
            elif val_losses[-1] > (best_val_loss + early_stop_delta):
                counter += 1
                if counter >= early_stop_patience:
                    print("Early stopping")
                    break

            #print(f"Epoch: {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}")
            # Print the actual number of epochs trained
            
            print(f"Step: {step}, Epoch: {epoch}\tTrain Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}")
            #### Lave plots med meshgrid f-funktion af normalising flow undervejs for at se ændringen
            
            model.train()
        
    print("Finished training.")
    return train_losses, val_losses, train_accuracies, val_accuracies, all_train_losses #,model

train_losses, val_losses, train_accuracies, val_accuracies, all_train_losses = train(postnet_model, optimiser, train_loader, test_loader, 
                                                                   num_epochs, validation_every_steps, early_stop_delta, early_stop_patience)

In [ ]:
# Plot loss of training and validation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(train_losses, label='Train Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_xlabel('Validation Epochs')
axes[0].set_ylabel('Loss')
axes[0].set_title('Train and Validation Loss')
axes[0].legend()

# Plot accuracies of training and validation
axes[1].plot(train_accuracies, label='Train Accuracy')
axes[1].plot(val_accuracies, label='Validation Accuracy')
axes[1].set_xlabel('Validation Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Train and Validation Accuracy')
axes[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# all train losses
print(len(all_train_losses))
# plot all_train_losses
plt.figure(figsize=(12,8))
plt.plot(all_train_losses,  '.',label='Train Loss', alpha=0.3)
#plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Train Loss')
plt.legend()
plt.show()

In [ ]:
# Load the best model from pth file type
device = torch.device("cpu")
flow_models = []
for class_label in range(num_classes):
    conditioner = Conditioner(in_dim=in_dim, out_dim=out_dim, num_hidden=num_hidden, hidden_dim=hidden_dim, num_params=num_params)
    affine_coupling = AffineCouplingLayer(conditioner, split=lambda x: x.chunk(2, dim=-1)) # split the tensor into 2 parts
    flows = [affine_coupling for _ in range(num_flows)]
    latent_distribution = torch.distributions.MultivariateNormal(loc=torch.zeros(data_dim).to(device), scale_tril=torch.eye(data_dim).to(device))

    flow_model = NormalisingFlow(latent_distribution, flows). to(device)
    #flow_model = NormalisingFlow(latent_distribution, flows).apply(init_weights).to(device)
    flow_models.append(flow_model)
best_model = PosteriorNetwork(latent_dim, flow_models, N, num_classes, y_train).to(device)
best_model.load_state_dict(torch.load('best_model_mnist.pth', map_location=device)['model_state_dict'])
#best_model.eval()

In [ ]:
# Test best model on test data
N = N.to(device) #move num classes to device just in case
alpha_values = []
# print(next(best_model.parameters()).is_cuda)
best_model.eval()
with torch.no_grad():
    for batch_index, (X_test_batch, y_test_batch) in enumerate(test_loader):
        X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
        # Forward pass to compute alpha values for this batch
        alpha_preds = best_model(X_test_batch, N) # batch x num_classes    
        # Append the alpha values to the list
        #print(alpha_preds.shape)
        alpha_values.append(alpha_preds)

# Combine alpha values from all batches list into a single tensor
alpha_values = torch.cat(alpha_values, dim=0)
print("Alpha values shape: ", alpha_values.shape)

# Normalize alpha values
alphas_norm = F.normalize(alpha_values, p=1, dim=1) #reduce to dim 1, sum to 1

In [ ]:
# Train the model
def train_flow_model(postnet_model, optimiser, num_epochs, X_train, y_train):
#def train_flow_model(flow_model, optimiser, X_train, num_epochs, class_label):
    losses = []
    #losses = torch.zeros(num_epochs)
    scheduler = optim.lr_scheduler.StepLR(optimiser, step_size=1000, gamma=0.999)
    for epoch in range(num_epochs):
        optimiser.zero_grad()
        log_prob = flow_model.log_prob(X_train)
        loss = -torch.mean(log_prob)
        loss.backward()
        utils.clip_grad_norm_(flow_model.parameters(), max_norm=5.0)  # Adjust max_norm as needed
        optimiser.step()
        scheduler.step()
        #losses[epoch] = loss.item()
        losses.append(loss.item())

        if (epoch + 1) % 100 == 0:
            print(f'Flow model {class_label} Epoch [{epoch + 1}/{num_epochs}], Loss: {loss:.4f}')
    return losses, flow_model 

# Training loop
for epoch in range(num_epochs):
    postnet_model.train()  # Set the model to training mode
    running_loss = 0.0  # Initialize the running loss
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data  # Get the input data and labels
        
        # Zero the parameter gradients
        optimiser.zero_grad()
        # Forward pass
        outputs = postnet_model(inputs, N, labels)  # Adjust this line according to your model's forward function
        # Compute the loss
        loss = loss(outputs, labels)  # Adjust this line according to your loss function
        # Backward pass
        loss.backward()
        # Update the model's parameters
        optimiser.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Print statistics every 'validation_every_steps' steps
        if i % validation_every_steps == validation_every_steps - 1:
            print(f"[Epoch {epoch + 1}, Step {i + 1}] Loss: {running_loss / validation_every_steps:.3f}")
            running_loss = 0.0

In [ ]:

### Training
for epoch in range(num_epochs):
    postnet_model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        optimiser.zero_grad()
        
        # Forward pass
        log_prob = postnet_model(images)
        # Compute the Posterior Network loss
        loss = posterior_network_loss(log_prob, labels)

        # Backpropagation
        loss.backward()
        optimiser.step()

        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

# Save the trained model if needed
torch.save(postnet_model.state_dict(), "posterior_net.pth")




In [ ]:
all_losses = []
flow_models_trained = []
for class_label in range(num_classes):
    print("class label",class_label)
    flow_models[class_label].apply(init_weights)
    losses, flow_model = train_flow_model(flow_models[class_label], optimisers[class_label], X_train[class_label], num_epochs, class_label)
    all_losses.append(losses)
    flow_models_trained.append(flow_model)
all_losses = np.array(all_losses)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].plot(all_losses[0,10:])
axes[0].set_title("Losses class 0")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[1].plot(all_losses[1,10:])
axes[1].set_title("Losses class 1")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Loss")
plt.tight_layout()
plt.show()

X_train = X_train.cpu().numpy()

In [ ]:
# Explore feature space
def plot_latent_space(flow_model, num_samples, class_label):
    z = flow_model.latent_sample(num_samples)
    x = flow_model.g(z)
    x = x.detach().cpu().numpy()
    z = z.detach().cpu().numpy()
    plt.figure(figsize=(8, 8))
    plt.scatter(z[:, 0], z[:, 1], c=class_label, cmap='tab10', alpha=0.5)
    plt.title('Latent space')
    plt.show()

    plt.figure(figsize=(8, 8))
    plt.scatter(x[:, 0], x[:, 1], c=class_label, cmap='tab10', alpha=0.5)
    plt.title('Feature space')
    plt.show()

In [ ]:
print("Final loss class 0: ", all_losses[0,-1])
print("Final loss class 1: ", all_losses[1,-1])

In [ ]:
# Generate samples from the trained models
num_samples_gen_per_class = 500
generated_samples = []
with torch.no_grad():
    for class_label in range(num_classes):
        samples = flow_models[class_label].sample(num_samples=num_samples_gen_per_class).cpu().numpy()
        generated_samples.append(samples)
generated_samples = np.array(generated_samples)
print("Generated samples shape: ", generated_samples.shape)

In [ ]:
# Check if class 0 from generated samples is different from class 1 in generated samples
# colors = ['r', 'b']
# plt.figure(figsize=(8, 6))
# for class_label in range(num_classes):
#     plt.scatter(generated_samples[class_label][:, 0], generated_samples[class_label][:, 1], 
#                 color=colors[class_label], alpha = 0.7, label=f'Generated Class {class_label}')
# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.legend()
# plt.title('Generated Samples for Class 0 and Class 1')
# plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Plot Class 0
axes[0].scatter(X_train[0, :, 0], X_train[0, :, 1], color='r', alpha=0.6, label='Class 0')
axes[0].scatter(generated_samples[0, :, 0], generated_samples[0, :, 1], color='g', alpha=0.6, label='Generated Class 0')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].set_title('Class 0 vs. Generated Class 0')
axes[0].legend()

# Plot Class 1
axes[1].scatter(X_train[1, :, 0], X_train[1, :, 1], color='b', alpha=0.6, label='Class 1')
axes[1].scatter(generated_samples[1, :, 0], generated_samples[1, :, 1], color='g', alpha=0.6, label='Generated Class 1')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].set_title('Class 1 vs. Generated Class 1')
axes[1].legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X_train[0, :, 0],X_train[0, :, 1], color='r', alpha=0.6, label='Class 0')
plt.scatter(generated_samples[0, :, 0], generated_samples[0, :, 1], color='g', alpha=0.5, label='Generated Class 0')
plt.scatter(X_train[1, :, 0], X_train[1, :, 1], color='b', alpha=0.6, label='Class 1')
plt.scatter(generated_samples[1, :, 0], generated_samples[1, :, 1], color='y', alpha=0.5, label='Generated Class 1')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Class 0 vs. Generated Class 0')
plt.legend()

In [ ]:
nx, ny = (100, 100)
x = np.linspace(-4, 4, nx)
y = np.linspace(-4, 4, ny)
xv, yv = np.meshgrid(x, y)
mesh = np.stack((xv.flatten(), yv.flatten()),axis=1)
mesh_tensor = torch.FloatTensor(mesh).to(device)
print(mesh_tensor.shape)

# class 0
logprobs = flow_models_trained[0].log_prob(mesh_tensor).clamp(min=-10, max=10)
logprobs = logprobs.cpu().reshape((100,100)).detach().numpy()
# class 1
logprobs2 = flow_models_trained[1].log_prob(mesh_tensor).clamp(min=-10, max=10)
logprobs2 = logprobs2.cpu().reshape((100,100)).detach().numpy()


# Plot the log probability as a heatmap
plt.figure(figsize=(8, 6))
plt.imshow(np.exp(logprobs), extent=[-4,4,-4,4], origin='lower', cmap='viridis')
plt.scatter(X_train[0, :, 0], X_train[0, :, 1], label='Original Data', alpha=0.07, color='red')
plt.colorbar(label='Probability Density')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Probability Density Function')
plt.show()


plt.figure(figsize=(8, 6))
plt.imshow(np.exp(logprobs2), extent=[-4,4,-4,4], origin='lower', cmap='viridis')
plt.scatter(X_train[1, :, 0], X_train[1, :, 1], label='Original Data', alpha=0.07, color='red')
plt.colorbar(label='Probability Density')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Probability Density Function')
plt.show()

#logprobs = flow_models[class_label].log_prob(X_tensor2)

# logprobs = logprobs.reshape((100,100))
#logprobs = logprobs.cpu().reshape((100,100)).detach().numpy()
# plt.figure(figsize = (8,8))
# plt.imshow(np.exp(logprobs), origin='lower', extent=[5,15,5,15])

# class_label = 1
# with torch.no_grad():
#     samples_c1 = flow_models[class_label].sample(num_samples=num_samples_gen_per_class).cpu().numpy()

# #plt.scatter(synthetic_data[:, 0],synthetic_data[:, 1], label='Generated Samples', alpha=0.05, color='yellow')
# plt.scatter(X_train[:, 0], X_train[:, 1], label='Original Data', alpha=0.07, color='red')
# plt.legend()
# plt.show()

In [ ]:
#scaler = StandardScaler()
#X_train = torch.stack([torch.FloatTensor(scaler.fit_transform(X_class)) for X_class in X_train])

# Data load
num_classes = 2
num_samples_per_class = 500
noise = 0.05
X_train, y_train = make_moons(n_samples=num_classes*num_samples_per_class, noise=noise)
X_train = torch.FloatTensor(np.array(X_train))
y_train = np.array(y_train)
# Multi class standardisation 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

#Plot the standardised dataset
# Plot the dataset
plt.figure(figsize=(5, 5))
colors = ['r', 'b']
for class_label in range(num_classes):
    plt.scatter(X_train[y_train == class_label][:, 0], 
                X_train[y_train == class_label][:, 1],
                color=colors[class_label], marker='o', label=f'Class {class_label}')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.title('Two Moons Dataset (Class 0 and Class 1)')
plt.show()

# class0 = X_train[y_train == 0]
# class1 = X_train[y_train == 1]
# X_train = torch.stack((class0, class1))
# print("X_train shape: ", X_train.shape)